<a href="https://colab.research.google.com/github/econdatatech/GISC420/blob/main/GISC_420_Mini_project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### GISC 420 T1 2022
# Mini Project Part 3


Corvin Idler,  ID 300598312 

This part of of the mini project experiments with the deck.gl / pydeck / WebGL stack

Again, first we've got to install a whole bunch of packages to make things work. 

It's a bit more complicated. Installing pydeck with pip upgrades all sorts of things, incl. jupyter which constitutes the colab runtine environement... 

**So one has to then manually restart the runtime before one can import all the packages after install. I took care of killing the runtime with code. It will restart automatically but one has to then manually execute the next cells after the one below. A RUN ALL CELLs will NOT work.**

In [ ]:
!pip install condacolab
!pip install jupyter_dash
!pip install dash_deck
!pip install flask-compress
!pip install pydeck
!pip install dash
!pip install dash-bootstrap-components
!pip install geopandas

#import condacolab 
#condacolab.install()
#!mamba install pydeck dash dash-bootstrap-components geopandas -c conda-forge
import os
os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 12.9 MB/s 
     |████████████████████████████████| 357 kB 59.2 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=451a631ddb18901273f1e4d4a731b3953998d81199956099a2df0198315a54bf
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.1 MB/s 
  Created wheel for dash-deck: filename=dash_deck-0.0.1-py3-none-any.whl size=2151467 sha256=e7992ed0957d2af9e2f1afb0163473ec0ae149243afd55dccca9d8a5b347641b
  Stored in directory: /root/.cache/pip/wheels/51/60/01/d340d4226c14497b1484676b1d0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 7.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 8.5 MB/s 
     |████████████████████████████████| 16.7 MB 49.6 MB/s 
     |████████████████████████████████| 6.3 MB 52.3 MB/s 


**When you reach this cell the runtime would have restarted. So a RUN ALL CELLS would have stopped here. You need to kick things off again**
Then we have to import a whole bunch of packages and load the export data from github



In [1]:
import dash 
from jupyter_dash import JupyterDash 

from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dash_deck
import pydeck as pdk
import pandas as pd
import requests
import geopandas as gpd
from io import BytesIO
import math

#just getting the data from my github repository
exports_geo=gpd.read_parquet(BytesIO(requests.get('https://github.com/econdatatech/GISC420/blob/main/nzexports_geo.parquet?raw=true', allow_redirects=True).content))



Now we can get going. I used code from the internet and amended it for my purpose.

In [2]:
#from https://dash.gallery/dash-deck-explorer/great-circle-layout
#Adapted itself from: https://pydeck.gl/gallery/great_circle_layer.html
df=exports_geo.loc[exports_geo['Year']=='2018',:]
df['source'] = df.apply(lambda x: [174.885971,-40.900557], axis=1)
df['destination'] = df.apply(lambda x: [x.geometry.x,x.geometry.y], axis=1)
df.loc[:,'Exports ($NZD fob)']=round(df['Exports ($NZD fob)']/1000000,2)
df.rename(columns = {'Exports ($NZD fob)':'Exports'}, inplace = True)
df['width'] = df.apply(lambda x:  math.ceil(x.Exports/500), axis=1)
df.sort_values('Exports', ascending=False,axis=0,inplace=True)

# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df.head(30),
    pickable=True,
    get_width='width',
    get_source_position='source',
    get_target_position='destination',
    get_source_color=[0, 0, 255],
    get_target_color=[255,0, 0],
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=0, longitude=0, zoom=0, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state,)
r.picking_radius = 10

r.to_html("line_layer.html")

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

<IPython.core.display.Javascript object>

'<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    <title>pydeck</title>\n        <script src="https://api.tiles.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>\n        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" />\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" />\n    <script src=\'https://cdn.jsdelivr.net/npm/@deck.gl/jupyter-widget@~8.5.*/dist/index.js\'></script>\n    <style>\n    body {\n  margin: 0;\n  padding: 0;\n  overflow: hidden;\n}\n\n#deck-map-container {\n  width: 100%;\n  height: 100%;\n  background-color: black;\n}\n\n#map {\n  pointer-events: none;\n  height: 100%;\n  width: 100%;\n  position: absolute;\n  z-index: 1;\n}\n\n#deckgl-overlay {\n  z-index: 2;\n  background: none;\n}\n\n#deck-map-wrapper {\n  width: 100%;\n  height: 100%;\n}\n\n#deck-container {\n  width: 100v

The above is a single HTML page. There is also the option to create a dash app that would allow for interactive dashboard functionality (like choosing reference years, etc). 

I didn't have time to implement e.g. a drop down menue for trading year and then issue a call back to redraw the map with adjusted data. But the principle is well demonstrated here https://github.com/plotly/jupyter-dash/blob/master/notebooks/getting_started.ipynb

At least I got the app going... unfortunately it (for now) just looks like the single HTML page from the cell above due to time constraints.

In [3]:
mapbox_api_token = 'pk.eyJ1IjoiaGFwcHlkZW1pZSIsImEiOiJjbDJxYXUwem8wM2k0M2JwZW4wNGVqZGlhIn0.0Jb40vvCEhRAY_v6y0YdfQ'
app = JupyterDash(__name__)
app.layout = html.Div(
    dash_deck.DeckGL(
        r.to_json(),
        id="deck-gl",
        mapboxKey=mapbox_api_token,
        tooltip={"text": "New Zealand to {Country}, {Exports} Mil $NZD Exports"}
    )
)
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

Both maps above took a while to get going, because the deck.gl team decided to go back and forth with the parameter for the thickness of the lines in the greatcircle layer (which inherits from the ArcLayer).

**First:**
Upgrading from deck.gl v5.2 to v5.3, https://deck.gl/docs/upgrade-guide#layer-properties

ArcLayer:	strokeWidth	(Removed Prop), getStrokeWidth (New Prop)



**Then shortly afterwards back to how it was:**

Upgrading from deck.gl v6.4 to v7.0, https://deck.gl/docs/upgrade-guide#layers-3

ArcLayer: getStrokeWidth (Removed Prop), getWidth (New Prop)


**Assessment:**
Overall the results are not too shabby. We got a color gradient and tooltip / mouseover functionality. But I'm still a bit bumped out, because variable line width isn't available (or at least I don't know how to).

There are people looking into it, but it would require to inject the line width information into the webgl shader... and that not something I can smarten up about in the time I have left :(

c.f.: https://github.com/visgl/deck.gl/issues/5021 and https://jsfiddle.net/symbolixau/09e2u5jg/191/

There are also people looking into gradient shaded polygons in WebGl c.f. https://github.com/visgl/deck.gl/issues/2663 and https://jsfiddle.net/symbolixau/on7432fj/70/

But again, to try and do that in pydeck is currently beyond the time scale of the assignement.
It appears that one might first have to write a SolidPolygon Layer based on the existing Polygon Layer https://pydeck.gl/gallery/polygon_layer.html
But I might be mislead or mistaken.. I just saw the author in above jsfiddle link did that. (c.f. as well https://deck.gl/docs/api-reference/layers/solid-polygon-layer)

**Drawback of the map:**


*   no variable line width
*   map snaps into place only after double click or pan with the mouse. Before the great circle lines are actually a bit out of position


#Custom Layer / External JS
With deck.gl there is the possibilty to create a custom layer using external javascript.
C.f. 

https://github.com/plotly/dash-deck#custom-layers

https://pydeck.gl/custom_layers.html

https://deck.gl/docs/developer-guide/custom-layers/layer-extensions

https://deck.gl/docs/developer-guide/custom-layers/writing-shaders

https://pydeck.gl/custom_layers.html

If ever one were to be able to get this going in a reasonable amount of time, one could make use of the existing flowmap.gl package which states that it can be used as a deck.gl custom layer. I simply didn't have enough time to get it going inside pydeck.
https://teralytics.github.io/flowmap.gl/index.html?path=/story/basic--basic-as-layer

https://github.com/FlowmapBlue/flowmap.gl/tree/main/examples/webpack-app/src

https://cdn.jsdelivr.net/npm/flowmap.gl@4.0.0/dist/

https://unpkg.com/flowmap.gl@4.0.0/dist/index.js

https://openbase.com/js/@flowmap.gl/core/documentation

https://github.com/FlowmapBlue/FlowmapBlue


#Other GL alternatives
There is the leaflet.gl package https://www.npmjs.com/package/@ksanek/leaflet.glify, https://github.com/robertleeplummerjr/Leaflet.glify that is available for folium (c.f. https://pypi.org/project/folium-glify-layer/) But again it would require me to read up on direct WebGL programming with too little time left.